In [1]:
import sys
sys.path.insert(0, sys.path[0].removesuffix('/src/jupyter_nb'))
sys.path.insert(1, sys.path[0] + '/src')
print(sys.path)
import pandas as pd
from src.robo import *
from src.utils import *
from src.calcEMA import *
from src.myenv import *
from src.train import *

tail = regression_times + 50
use_cols = ['open_time', 'close']

estimator = 'lightgbm'

['/home/marcelo/des/mg_crypto_trader', '/home/marcelo/des/mg_crypto_trader/src', '/home/marcelo/des/mg_crypto_trader/src/jupyter_nb', '/usr/lib64/python310.zip', '/usr/lib64/python3.10', '/usr/lib64/python3.10/lib-dynload', '', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib/python3.10/site-packages', '/home/marcelo/des/mg_crypto_trader/.env/lib64/python3.10/site-packages/IPython/extensions', '/home/marcelo/.ipython']


In [ ]:
symbol = 'BTCUSDT'
experiment, model = load_model(symbol, estimator)  # cassification_experiment

In [ ]:
df = get_data(symbol, save_database=False, interval='1h', tail=tail, adjust_index=True)
df.to_csv('log_after_1st_get_data.csv', sep=';', index=False)
df

In [ ]:
df = calc_RSI(df)
df.to_csv('log_after_1st_calc_RSI.csv', sep=';', index=False)
df

In [ ]:
df.dropna(inplace=True)
df

In [ ]:
numeric_features = ['close', 'rsi']
df, _ = regresstion_times(df, numeric_features, regression_times)
df.to_csv('log_after_1st_regresstion_times.csv', sep=';')
df

In [ ]:
df.info()

In [ ]:
window = 14
close_price = 'close'
count = df.shape[0]
df.iloc[count-1:count]['rsi'] = 0


#print(df[['close', 'rsi']].tail())
# _df = df.tail(window + 10)[[close_price, 'rsi']].copy()
_df = df[[close_price, 'rsi']].tail(window+50).copy()
_count = _df.shape[0]
#print(_df)
_df['change'] = _df[close_price].diff()
_df['gain'] = _df.change.mask(_df.change < 0, 0.0)
_df['loss'] = -_df.change.mask(_df.change > 0, -0.0)
_df['avg_gain'] = rma(_df.gain.to_numpy(), window)
_df['avg_loss'] = rma(_df.loss.to_numpy(), window)

_df['rs'] = _df.avg_gain / _df.avg_loss
_df['rsi'] = 100 - (100 / (1 + _df.rs))
#print(_df)

#print(df.iloc[count-1:count]['rsi'])
#print(_df.iloc[_count-1:_count]['rsi'])

df.update(_df.iloc[_count-1:_count]['rsi'])

df.tail()

In [ ]:
cont = 0

In [ ]:
#while True:

# df = get_data(symbol, save_database=False, interval='1h', tail=tail)
_df = get_klines(symbol, interval='1h', max_date=None, limit=1, adjust_index=True)
_df.to_csv(f'log_after_{cont}_get_klines.csv', sep=';', index=False)

#df = pd.concat([df, _df]).drop_duplicates(keep='last')
if _df.index.isin(df.index):
    df.update(_df)
else:
    df = pd.concat([df, _df])   
df.to_csv(f'log_after_{cont}_merge.csv', sep=';', index=False)

df = calc_RSI(df, last_one=True)
df.to_csv(f'log_after_{cont}_calc_RSI.csv', sep=';', index=False)

df, _ = regresstion_times(df, numeric_features, regression_times, last_one=True)
df.to_csv(f'log_after_{cont}_regresstion_times.csv', sep=';', index=False)

df[['open_time'] + numeric_features].to_csv('log_experiment_data.log', sep=';', index=False)
df_predict = experiment.predict_model(model, df.tail(1))
df_predict.to_csv(f'log_after_{cont}_df_predict.csv', sep=';', index=False)

operacao = df_predict['prediction_label'].values[0]
if (operacao.startswith('SOBE') or operacao.startswith('CAI')):
    send_message(df_predict)
time.sleep(sleep_refresh)
cont += 1

In [ ]:
import pandas as pd
df1 = pd.DataFrame({'open_time': [1, 55, 32, 234], 'close': [3, 4, 5, 6], 'C': [5, 6, 34, 5], 'D': [7, 8, 54, 66], 'label': ['A', 'A', 'B', 'B']}, index=[0,1,2,3])
df1

In [ ]:
df2 = pd.DataFrame({'open_time': [55], 'close': [77]}, index=[1])
df2

In [ ]:
i = df2.iloc[df2.shape[0]-1:df2.shape[0]].index
i

In [ ]:
df2.index.isin(df1.index)

In [ ]:
_df = get_klines('BTCUSDT', interval='1h', max_date=None, limit=200, adjust_index=True)
_df

In [ ]:
from pycaret.classification.oop import ClassificationExperiment

ca = ClassificationExperiment()
setup = ca.setup(_df, target='symbol')

In [ ]:
best = setup.compare_models()

best

In [ ]:
setup.pull().index[0]

In [ ]:
import datetime 
datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

In [ ]:
import pandas as pd
import datetime
import os

df_resultado_simulacao = pd.DataFrame()
if (os.path.exists('resultado_simulacao.csv')):
    df_resultado_simulacao = pd.read_csv('resultado_simulacao.csv', sep=';')

result_simulado = {}
result_simulado['model_name'] = 'BTCUSDT_lr_SL_2.0_RT_1440_RPL_24_1'
result_simulado['data'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
result_simulado['symbol'] = 'BTCUSDT'
result_simulado['estimator'] = 'lightgbm'
result_simulado['train_size'] = 0.7 
result_simulado['start_train_date'] = '2023-06-01'
result_simulado['start_test_date'] = '2023-08-08'
result_simulado['regression_times'] = 24*30
result_simulado['regression_profit_and_loss'] = 24
result_simulado['stop_loss'] = 2.0
result_simulado['fold'] = 3
result_simulado['saldo_inicial'] = 100.00
result_simulado['saldo_final'] = 91.87

print(result_simulado)

df = pd.DataFrame.from_dict([result_simulado])

df_resultado_simulacao = pd.concat([df_resultado_simulacao, df], ignore_index=True)
df_resultado_simulacao.sort_values('saldo_final', inplace=True)

df_resultado_simulacao.to_csv('resultado_simulacao.csv', sep=';', index=False)

In [ ]:
from binance.client import Client
client = Client()
klines = client.get_historical_klines(symbol='BTCUSDT', interval='1h', start_str='2010-01-01', limit=1000)



In [ ]:
import pandas as pd


all_klines_cols = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume',
                   'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore']

columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'quote_asset_volume', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'number_of_trades', 'ignore', 'close_time']

df_klines = pd.DataFrame(data=klines, columns=all_klines_cols)[columns]
df_klines['open_time'] = pd.to_datetime(df_klines['open_time'], unit='ms')
df_klines['close_time'] = pd.to_datetime(df_klines['close_time'], unit='ms')

In [ ]:
df_klines

In [2]:
download_data()


get_database: name:  /home/marcelo/des/mg_crypto_trader/src/data/BTCUSDT/BTCUSDT.csv
get_database: columns:  ['open_time', 'close']
get_database: count_rows: 52910 - symbol: BTCUSDT - tail: -1 - adjust_index: False
get_database: duplicated: 0
get_data: Downloading data for symbol: BTCUSDT


TypeError: '>=' not supported between instances of 'Timestamp' and 'int'